# Delta experiments

## Import libraries

In [1]:
import polars as pl
from deltalake import DeltaTable, write_deltalake

delta_path = "../data/test_delta"

# Create a mock Polars DataFrame and save it as a Delta. It should have columns date (in yyyy-mm-dd format), country, and value.
df = pl.DataFrame({
    "date": ["2023-01-01", "2023-01-02", "2023-01-03"],
    "country": ["US", "CA", "MX"],
    "value": [100, 200, 300],
})

# Write Delta using delta-rs write_deltalake function
write_deltalake(
    table_or_uri=delta_path,
    data=df.to_arrow(),
    mode="overwrite",
    name="test_delta",
    partition_by=["date"],
    schema_mode="merge",
)

One can also use `write_delta` predicate from Polars

In [2]:
df.write_delta(
    target=delta_path,
    mode="overwrite",
    delta_write_options={
        "partition_by": ["date"],
        "schema_mode": "merge",
        "name": "test_delta",
    },
)

## Read Delta

We can read delta using the `DeltaTable` class, and then converting to Polars through Arrow (but that will load the whole table with no possibility of filtering data), or using `scan_delta` and filter before materializing it.

In case the Delta is located in a remote storage, we can pass `storage_options` parameter (to both Polars and DeltaTable).

In [3]:
dt = DeltaTable(delta_path)
pl.from_arrow(dt.to_pyarrow_table()).select("date", "country", "value").sort("date")

date,country,value
str,str,i64
"""2023-01-01""","""US""",100
"""2023-01-02""","""CA""",200
"""2023-01-03""","""MX""",300


In [4]:
pl.scan_delta(delta_path).collect().select("date", "country", "value").sort("date")

date,country,value
str,str,i64
"""2023-01-01""","""US""",100
"""2023-01-02""","""CA""",200
"""2023-01-03""","""MX""",300


## Read Delta using DuckDB

In [5]:
import duckdb

# Query the Delta table using DuckDB
duckdb.sql(f"""
SELECT date, country, value FROM delta_scan('{delta_path}' ORDER BY date)
           """)


┌────────────┬─────────┬───────┐
│    date    │ country │ value │
│  varchar   │ varchar │ int64 │
├────────────┼─────────┼───────┤
│ 2023-01-02 │ CA      │   200 │
│ 2023-01-03 │ MX      │   300 │
│ 2023-01-01 │ US      │   100 │
└────────────┴─────────┴───────┘

## Upsert

Create another DataFrame with a 2 rows, one that will be appended and one that will be overwritten (considering `date` as join column).

In [6]:
upsert_df = pl.DataFrame({
    "date": ["2023-01-01", "2023-01-04"],
    "country": ["US", "IT"],
    "value": [150, 250],
})
upsert_df

date,country,value
str,str,i64
"""2023-01-01""","""US""",150
"""2023-01-04""","""IT""",250


In [7]:
# Upsert the new data into the Delta table
dt.merge(
    source=upsert_df.to_arrow(),
    source_alias="source",
    target_alias="target",
    predicate="source.date = target.date",
    merge_schema=True,
).when_matched_update_all().when_not_matched_insert_all().execute()


{'num_source_rows': 2,
 'num_target_rows_inserted': 1,
 'num_target_rows_updated': 1,
 'num_target_rows_deleted': 0,
 'num_target_rows_copied': 0,
 'num_output_rows': 2,
 'num_target_files_scanned': 3,
 'num_target_files_skipped_during_scan': 0,
 'num_target_files_added': 2,
 'num_target_files_removed': 1,
 'execution_time_ms': 10,
 'scan_time_ms': 0,
 'rewrite_time_ms': 1}

In [8]:
# Verify the upsert
pl.scan_delta(delta_path).collect().select("date", "country", "value").sort("date")


date,country,value
str,str,i64
"""2023-01-01""","""US""",150
"""2023-01-02""","""CA""",200
"""2023-01-03""","""MX""",300
"""2023-01-04""","""IT""",250


## Read Delta History

In [9]:
pl.from_dicts(dt.history()).sort("timestamp")

timestamp,operation,operationParameters,readVersion,clientVersion,operationMetrics,version
i64,str,struct[6],i64,str,struct[17],i64
1749397303868,"""WRITE""","{null,null,null,null,""[""date""]"",""Overwrite""}",null,"""delta-rs.py-0.25.5""","{2,null,null,null,null,null,null,null,null,null,null,null,null,3,3,0,0}",0
1749397303879,"""WRITE""","{null,null,null,null,""[""date""]"",""Overwrite""}",null,"""delta-rs.py-0.25.5""","{2,null,null,null,null,null,null,null,null,null,null,null,null,3,3,0,3}",1
1749397304078,"""MERGE""","{""[]"",""[{""actionType"":""insert""}]"",""source.date = target.date"",""[{""actionType"":""update""}]"",null,null}",1,"""delta-rs.py-0.25.5""","{10,2,2,2,1,3,0,0,0,1,1,1,0,null,null,null,null}",2


## Perform Time Travel, Vacuum and Optimize

Delta has some nice features:

- **Time travel**: Restore old version
- **Vacuum**: Remove data not referenced by available table version
- **Optimize**: Compact smaller files together

In [10]:
# Load initial version from Polars
pl.scan_delta(delta_path, version=0).collect().select("date", "country", "value").sort(
    "date"
)

date,country,value
str,str,i64
"""2023-01-01""","""US""",100
"""2023-01-02""","""CA""",200
"""2023-01-03""","""MX""",300


In [11]:
# Restore initial version
dt.restore(target=0)
pl.scan_delta(delta_path).collect().select("date", "country", "value").sort("date")

date,country,value
str,str,i64
"""2023-01-01""","""US""",100
"""2023-01-02""","""CA""",200
"""2023-01-03""","""MX""",300


However, restoring a Delta causes DuckDB to break on read.

In [12]:
import duckdb

# Query the Delta table using DuckDB
duckdb.sql(f"""
SELECT * FROM delta_scan('{delta_path}')
           """)


IOException: IO Error: DeltKernel InternalError (31): Internal error Reader features must not be present when minimum reader version = 1 or 2. This is a kernel bug, please report.

In [ ]:
# Restore latest version
print(dt.restore(target=1))
# Verify the restored version
pl.scan_delta(delta_path).collect().select("date", "country", "value").sort("date")

{'numRemovedFile': 3, 'numRestoredFile': 3}


date,country,value
str,str,i64
"""2023-01-01""","""US""",100
"""2023-01-02""","""CA""",200
"""2023-01-03""","""MX""",300


The following command will remove all files that are not referenced by Delta versions in history. It will output a list of deleted files, that are not restorable.

A `retention_hours` period can be specified, along with `dry_run`. `enforce_retention_duration` needs to be set at False in case `retention_hours` is lower than the Delta default one, which is 168 hours in case not specified differently at Delta creation with `configs` param.

In [ ]:
# Vacuum Delta
dt.vacuum(retention_hours=0, dry_run=True, enforce_retention_duration=False)

['date=2023-01-03/part-00001-a1b7bf07-ab81-42c2-a49e-ea55199794e1-c000.snappy.parquet',
 'date=2023-01-02/part-00001-37009a44-ff28-4a00-9e38-3298ddd58c85-c000.snappy.parquet',
 'date=2023-01-01/part-00001-e290a568-4289-43ab-b764-a77ee1fc0e25-c000.snappy.parquet']

*Optimize* will compact smaller files (think about small insertion) together, so that queries can be speed up by looking at less files.

In [ ]:
dt.optimize.compact()

{'numFilesAdded': 0,
 'numFilesRemoved': 0,
 'filesAdded': '{"avg":0.0,"max":0,"min":0,"totalFiles":0,"totalSize":0}',
 'filesRemoved': '{"avg":0.0,"max":0,"min":0,"totalFiles":0,"totalSize":0}',
 'partitionsOptimized': 0,
 'numBatches': 0,
 'totalConsideredFiles': 3,
 'totalFilesSkipped': 3,
 'preserveInsertionOrder': True}

## Z-Order

Z-Order is a way to reorganize data in storage in order to optimize queries. It allows for more data-skipping by colocating relevant files together. Think about it a smart sorting of files based on one or multiple columns

In [ ]:
dt.optimize.z_order(columns=["country"])

{'numFilesAdded': 3,
 'numFilesRemoved': 3,
 'filesAdded': '{"avg":796.0,"max":796,"min":796,"totalFiles":3,"totalSize":2388}',
 'filesRemoved': '{"avg":766.0,"max":766,"min":766,"totalFiles":3,"totalSize":2298}',
 'partitionsOptimized': 0,
 'numBatches': 3,
 'totalConsideredFiles': 3,
 'totalFilesSkipped': 0,
 'preserveInsertionOrder': True}

## Unity Catalog experiments

In [5]:
import polars as pl

c = pl.Catalog(workspace_url="http://localhost:8080", require_https=False)
c.list_catalogs()

[CatalogInfo(name='unity', comment='Main catalog', properties={}, options={}, storage_location=None, created_at=datetime.datetime(2024, 7, 17, 18, 40, 5, 334000, tzinfo=datetime.timezone.utc), created_by=None, updated_at=None, updated_by=None)]

In [9]:
c.list_tables(catalog_name="unity", namespace="default")

[TableInfo(name='marksheet', comment='Managed table', table_id='c389adfa-5c8f-497b-8f70-26c2cca4976d', table_type='MANAGED', storage_location='file:///home/unitycatalog/etc/data/managed/unity/default/tables/marksheet/', data_source_format='DELTA', columns=[ColumnInfo(name='id', type_name='INT', type_text='int', type_json='{"name":"id","type":"integer","nullable":false,"metadata":{}}', position=0, comment='ID primary key', partition_index=None), ColumnInfo(name='name', type_name='STRING', type_text='string', type_json='{"name":"name","type":"string","nullable":false,"metadata":{}}', position=1, comment='Name of the entity', partition_index=None), ColumnInfo(name='marks', type_name='INT', type_text='int', type_json='{"name":"marks","type":"integer","nullable":true,"metadata":{}}', position=2, comment='Marks of the entity', partition_index=None)], properties={'key2': 'value2', 'key1': 'value1'}, created_at=datetime.datetime(2024, 7, 17, 18, 40, 5, 595000, tzinfo=datetime.timezone.utc), cr

In [13]:
# mock dataframe
df = pl.DataFrame({
    "date": ["2023-01-01", "2023-01-02", "2023-01-03"],
    "country": ["US", "CA", "MX"],
    "value": [100, 200, 300],
})

In [ ]:
delta_path = ""  # replace with path of choice
c.create_table(
    catalog_name="unity",
    namespace="default",
    table_name="test_table",
    schema=df.schema,
    table_type="EXTERNAL",
    data_source_format="DELTA",
    storage_root=delta_path,
)

In [21]:
c.write_table(
    df=df,
    catalog_name="unity",
    namespace="default",
    delta_mode="overwrite",
    table_name="test_table",
)

In [ ]:
c.scan_table(
    catalog_name="unity", namespace="default", table_name="test_table"
).collect()

date,country,value
str,str,i64
"""2023-01-01""","""US""",100
"""2023-01-02""","""CA""",200
"""2023-01-03""","""MX""",300


In [ ]:
import duckdb

duckdb.sql("""
INSTALL uc_catalog;
INSTALL delta;
LOAD delta;
LOAD uc_catalog;
CREATE OR REPLACE SECRET uc_secret (
	TYPE UC,
	ENDPOINT 'http://127.0.0.1:8080',
    TOKEN 'not used',
    AWS_REGION 'eu-central-1'

);
ATTACH IF NOT EXISTS 'unity' AS test_catalog (TYPE UC_CATALOG, SECRET uc_secret);
SHOW ALL TABLES;
""")

┌──────────┬─────────┬─────────┬──────────────┬──────────────┬───────────┐
│ database │ schema  │  name   │ column_names │ column_types │ temporary │
│ varchar  │ varchar │ varchar │  varchar[]   │  varchar[]   │  boolean  │
├──────────┴─────────┴─────────┴──────────────┴──────────────┴───────────┤
│                                 0 rows                                 │
└────────────────────────────────────────────────────────────────────────┘